In [1]:
#source("http://cf.10xgenomics.com/supp/cell-exp/rkit-install-2.0.0.R")

In [2]:
library(Matrix)
library(cellrangerRkit)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:Matrix’:

    colMeans, colSums, rowMeans, rowSums, which

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply,

In [3]:
getSimuResults = function(simu_name){
    print(paste0("~/jamboree/task1_simu/",simu_name,".gz"))
    tab <- read.table(gzfile(paste0("~/jamboree/task1_simu/",simu_name,".gz")), header=T, sep = ",")

    m = sparseMatrix(i = as.numeric(tab$row), j = as.numeric(tab$column), x = tab$value)

    rownames(m) = levels(tab$row)
    colnames(m) = levels(tab$column)

    fd = data.frame(id = rownames(m),row.names = rownames(m))
    pd = data.frame(id = colnames(m),row.names = colnames(m))

    gbc_obj_all = newGeneBCMatrix(mat = m,fd = fd,pd = pd)

    # Ignore undetected molecules
    gbc_obj = gbc_obj_all[rowSums(exprs(gbc_obj_all)) > 0,]
    rm(list = "gbc_obj_all")
    
    ## ----- Total UMI Counts -----
    umi_counts = colSums(exprs(gbc_obj))

    # Filter unassociated barcodes
    gbc_obj_associated = gbc_obj[,umi_counts > 0]
    rm(list = "gbc_obj")
    umi_counts = umi_counts[umi_counts > 0]

    amb_thresh = 100

    # Compute Ambient Multinomial Sampling Probabilities
    is_amb = umi_counts < amb_thresh
    print("is_amb:")
    print(table(is_amb))
    amb_counts = rowSums(exprs(gbc_obj_associated)[,is_amb])
    amb_alpha = amb_counts/sum(amb_counts)

    # Compare to Non-Ambient Sampling Probabilities
    nonamb_counts = rowSums(exprs(gbc_obj_associated)[,!is_amb])
    nonamb_alpha = nonamb_counts/sum(nonamb_counts)

    nhigh = 100
    high_thresh = sort(amb_alpha,decreasing = TRUE)[nhigh]

    ## ----- Select High Genes ----
    amb_alpha_high = amb_alpha[amb_alpha >= high_thresh]
    amb_alpha_high = amb_alpha_high/sum(amb_alpha_high)
    gbc_obj_high = gbc_obj_associated[amb_alpha >= high_thresh,]
    rm(list = c("gbc_obj_associated","amb_alpha","nonamb_alpha"))

    ## ----- Deviation from Mean -----
    gbc_obj_high_nonamb = gbc_obj_high[,!is_amb]
    umi_counts_nonamb = umi_counts[!is_amb]
    binsize = 100
    bin_val = cut(seq_len(ncol(gbc_obj_high_nonamb)),
              floor(ncol(gbc_obj_high_nonamb)/binsize),
              labels = FALSE)[order(order(umi_counts_nonamb,decreasing = TRUE))]
    bin_alpha = sapply(seq_len(max(bin_val)),function(s){
      #print(s/max(bin_val))
      bin_counts = rowSums(exprs(gbc_obj_high_nonamb)[,bin_val == s])
      bin_counts/sum(bin_counts)
    })

    diff_alpha = sqrt(colMeans((bin_alpha - amb_alpha_high)^2))
    bin_trans = which( diff_alpha == max(diff_alpha))
    is_cell = umi_counts > max(umi_counts_nonamb[bin_val == bin_trans])
    rm(list = ls()[!ls() %in% c("is_cell","getSimuResults")])
    return(is_cell)
}

In [9]:
noms = gsub(".gz|.*.txt","",list.files("~/jamboree/task1_simu/"))
noms = noms[grepl("simu",noms)]

is_cell_out = sapply(noms,getSimuResults)

[1] "~/jamboree/task1_simu/simu_1.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 72315 456761 
[1] "~/jamboree/task1_simu/simu_10.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 72995 461879 
[1] "~/jamboree/task1_simu/simu_11.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 73878 468994 
[1] "~/jamboree/task1_simu/simu_12.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 75686 482991 
[1] "~/jamboree/task1_simu/simu_13.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 75287 457278 
[1] "~/jamboree/task1_simu/simu_14.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 78440 459576 
[1] "~/jamboree/task1_simu/simu_15.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 84814 464360 
[1] "~/jamboree/task1_simu/simu_16.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 97698 474069 
[1] "~/jamboree/task1_simu/simu_2.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 72522 458722 
[1] "~/jamboree/task1_simu/simu_3.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 72905 462409 
[1] "~/jamboree/task1_simu/simu_4.gz"
[1] "is_amb:"
is_amb
 FALSE   TRUE 
 73800 470133 
[1] "~/jambore

In [20]:
sapply(noms,function(simu_name){
    print(simu_name)
    is_cell = is_cell_out[[simu_name]]
    good_codes = names(which(is_cell))
    fn = paste0("~/shared_scratch/group3/group3_task1_DS_1_cell_barcodes_",gsub(".*_","simu",simu_name),"_ambpropdev.tab")
    df = cbind(good_codes,good_codes)
    colnames(df) = c("Cell Barcode","Source Barcode")
    write.table(file = fn, df, sep = "\t",quote = FALSE,row.names = FALSE)
})


[1] "simu_1"
[1] "simu_10"
[1] "simu_11"
[1] "simu_12"
[1] "simu_13"
[1] "simu_14"
[1] "simu_15"
[1] "simu_16"
[1] "simu_2"
[1] "simu_3"
[1] "simu_4"
[1] "simu_5"
[1] "simu_6"
[1] "simu_7"
[1] "simu_8"
[1] "simu_9"


$simu_1
NULL

$simu_10
NULL

$simu_11
NULL

$simu_12
NULL

$simu_13
NULL

$simu_14
NULL

$simu_15
NULL

$simu_16
NULL

$simu_2
NULL

$simu_3
NULL

$simu_4
NULL

$simu_5
NULL

$simu_6
NULL

$simu_7
NULL

$simu_8
NULL

$simu_9
NULL


In [23]:
simu_name = "simu_1"
print(simu_name)
is_cell = is_cell_out[[simu_name]]
good_codes = names(which(is_cell))

[1] "simu_1"
